In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

from sklearn.model_selection import train_test_split
from ordinal_regression_model.utils_process_ordinal_variable import convert_target
from ordinal_regression_model.sklearn_ordianl_logit_model import OrdinalLogitClassifier
# scikit-learn API wrapper for ordinal logistic regression model

# Tutorial:

- Ordinal Regression, *Statsmodels Document*, [site](https://www.statsmodels.org/stable/examples/notebooks/generated/ordinal_regression.html)

- Ordinal Logistic Regression, R Data Analysis Example, *UCLA Statistical Methods and Data Analysis*, [site](https://stats.oarc.ucla.edu/r/dae/ordinal-logistic-regression/)


# 1. Data preparation

In [2]:
def print_data_info(X, Y_num, Y_ord):
    print('\nData shape:', X.shape, Y_ord.shape,
        '\nData types:',
        '\n\tNumeric data:', type(Y_num), Y_num.dtypes,
        '\n\tOrdinal data:', type(Y_ord), Y_ord.dtypes,
        '\nOrdinal target variable:',
        '\n\tDistribution:', Y_ord.value_counts().sort_index().to_dict(),
        '\n\tIs ordered:', Y_ord.cat.ordered,
        '\n\tOrdered labels:', Y_ord.cat.categories,
        '\nTarget values:\n', pd.concat([Y_ord, Y_num], axis=1).drop_duplicates(ignore_index=True)
    )

In [3]:
# Wine quality dataset
from ordinal_regression_model.load_dataset import load_dataset
data_X, data_Y_num = load_dataset('wine_quality', preprocess=True, split_target=True)

# Reclassify numerical target into ordinal target
data_Y_num = data_Y_num.map(
    {3 : 'Low', 4 : 'Low', 5 : 'Low', 6 : 'Medium', 7 : 'Medium', 8 : 'High', 9 : 'High'})

# Convert numerical variable into ordinal variable
data_Y_ord = convert_target(
    data_Y_num, object_type='ordinal', mapper={'Low' : 1, 'Medium' : 2, 'High' : 3},
    to_numeric=True)

# Print data information
print_data_info(data_X, data_Y_num, data_Y_ord)


Data shape: (6497, 12) (6497,) 
Data types: 
	Numeric data: <class 'pandas.core.series.Series'> object 
	Ordinal data: <class 'pandas.core.series.Series'> category 
Ordinal target variable: 
	Distribution: {1: 2384, 2: 3915, 3: 198} 
	Is ordered: True 
	Ordered labels: Index([1, 2, 3], dtype='int64') 
Target values:
   quality quality
0       2  Medium
1       1     Low
2       3    High


In [4]:
data_Y = convert_target(data_Y_ord, object_type='label')
print(data_Y.dtype)

# split data into train and test sets
train_idx, test_idx = train_test_split(range(len(data_X)), test_size=0.4, random_state=42, stratify=data_Y_ord)

train_X = data_X.iloc[train_idx, :]
train_Y = data_Y.iloc[train_idx]
test_X = data_X.iloc[test_idx, :]
test_Y = data_Y.iloc[test_idx]

print('\nShape of train data:', train_X.shape, train_Y.shape,
      '\nShape of test data:', test_X.shape, test_Y.shape,
      '\nDistribution of train target variables:', train_Y.value_counts(dropna=False).sort_index().to_dict(),
      '\nDistribution of test target variables:', test_Y.value_counts(dropna=False).sort_index().to_dict())

int8

Shape of train data: (3898, 12) (3898,) 
Shape of test data: (2599, 12) (2599,) 
Distribution of train target variables: {0: 1430, 1: 2349, 2: 119} 
Distribution of test target variables: {0: 954, 1: 1566, 2: 79}


# 2. Ordinal regression model

#### (1) MNLogit model

In [5]:
model_1 = OrdinalLogitClassifier(
    ordinal_target = False,
    method='bfgs', gtol=1e-7, maxiter=1000)

model_1.fit(train_X, train_Y)
data_pred = model_1.predict(train_X)
data_pred_prob = model_1.predict_proba(train_X)

print('model instance:', model_1.model_)

Optimization terminated successfully.
         Current function value: 0.623221
         Iterations: 214
         Function evaluations: 220
         Gradient evaluations: 220
model instance: <statsmodels.discrete.discrete_model.MNLogit object at 0x0000029831FAE800>


#### (2) Ordered logit model

In [6]:
model_2 = OrdinalLogitClassifier(
    ordinal_target = True, ordinal_labels = [0, 1, 2], distribution = 'logit',
    method='bfgs', gtol=1e-7, maxiter=1000)

model_2.fit(train_X, train_Y)
data_pred = model_2.predict(train_X)
data_pred_prob = model_2.predict_proba(train_X)

print('model instance:', model_2.model_)

Optimization terminated successfully.
         Current function value: 0.625748
         Iterations: 124
         Function evaluations: 130
         Gradient evaluations: 130
model instance: <statsmodels.miscmodels.ordinal_model.OrderedModel object at 0x00000298324074C0>


#### (3) Ordered probit model

In [7]:
model_3 = OrdinalLogitClassifier(
    ordinal_target = True, ordinal_labels = [0, 1, 2], distribution = 'probit',
    method='bfgs', gtol=1e-7, maxiter=1000)

model_3.fit(train_X, train_Y)
data_pred = model_3.predict(train_X)
data_pred_prob = model_3.predict_proba(train_X)

print('model instance:', model_3.model_)

Optimization terminated successfully.
         Current function value: 0.627857
         Iterations: 106
         Function evaluations: 110
         Gradient evaluations: 110
model instance: <statsmodels.miscmodels.ordinal_model.OrderedModel object at 0x0000029832406650>


# 3. Performance evaluation

In [8]:
from ordinal_regression_model.metrics import classification_metric, ranked_probability_score

model_name_li = ['MNLogit', 'OrderedLogit', 'OrderedProbit']

metrics_all = {}
for _name, model in zip(model_name_li, [model_1, model_2, model_3]):
    print('\nModel instance:', model.model_)
    metrics_all[_name] = {}

    for data in ['train', 'test']:
        X_input = train_X if data == 'train' else test_X
        Y_input = train_Y if data == 'train' else test_Y
        pred = model.predict(X_input)
        pred_proba = model.predict_proba(X_input)

        metrics_all[_name][data] = classification_metric(Y_input, pred, pred_proba, sample_weight='balanced')

# convert the three demensional dictionary into a two dimensional dictionary
metrics = pd.concat({k : pd.DataFrame(v).T.unstack() for k, v in metrics_all.items()}, axis=1)
metrics.round(3)


Model instance: <statsmodels.discrete.discrete_model.MNLogit object at 0x0000029831FAE800>

Model instance: <statsmodels.miscmodels.ordinal_model.OrderedModel object at 0x00000298324074C0>

Model instance: <statsmodels.miscmodels.ordinal_model.OrderedModel object at 0x0000029832406650>


C:\ProgramData\mambaforge\envs\py310geo\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


MNLogit  OrderedLogit  OrderedProbit
precision                         train    0.468         0.468          0.468
                                  test     0.470         0.467          0.469
recall                            train    0.470         0.468          0.466
                                  test     0.468         0.463          0.461
roc_auc                           train    0.793         0.788          0.788
                                  test     0.789         0.784          0.785
accuracy                          train    0.713         0.711          0.711
                                  test     0.713         0.709          0.709
balanced_accuracy                 train    0.470         0.468          0.466
                                  test     0.468         0.463          0.461
f1_macro                          train    0.467         0.466          0.464
                                  test     0.466         0.461          0.460
cohen_kappa                       train    0.392         0.388          0.384
                                  test     0.389         0.376          0.373
log_loss                          train    0.623         0.626          0.628
                                  test     0.635         0.637          0.640
balanced_log_loss                 train    1.344         1.376          1.372
                                  test     1.342         1.361          1.351
ranked_probability_score          train    0.199         0.199          0.200
                                  test     0.202         0.203          0.203
balanced_ranked_probability_score train    0.424         0.426          0.420
                                  test     0.425         0.428          0.421

In [9]:
class OrdinalRegression():
    def __init__(self, endog, exog, distr='probit'):
        '''

        :param endog:
            endogenous variables, aka dependent, response, target, or y
        :param exog:
            exogenous variables, aka independent, features, predictors, or X
        :param distr:
            distribution to use, either 'probit' or 'logit'
        '''

        self.endog = endog.values
        self.exog = exog.values
        self.distr = distr

        self.nobs = self.exog.shape[0]
        self.n_params = self.exog.shape[1]
        # number of categories in the endogenous variable
        self.n_cats = len(np.unique(self.endog))

        # random initialization of parameters
        self.params = 1 - np.random.rand(self.n_params)
        self.thresholds = 1 - np.random.rand(self.n_cats - 1)

        self.exog_names = exog.columns.to_list()
        self.endog_name = endog.name
    # ----------------------------------------------------------

    def fit(self, method='bfgs', maxiter=100, tol=1e-6):
        if self.distr == 'probit':
            self.model = OrderedModel(self.endog, self.exog, distr='probit')
        elif self.distr == 'logit':
            self.model = OrderedModel(self.endog, self.exog, distr='logit')
        else:
            raise ValueError("Unknown distribution")

        self.results = self.model.fit(method=method, maxiter=maxiter, tol=tol)
        return self.results
    # ----------------------------------------------------------
    def probability(self):

    # ----------------------------------------------------------
    def probability_obs(self, params):
        '''
        Log-likelihood of OrderdModel for all observations.
        :return:
        '''
        mat_wx = np.dot(self.exog, params)

        return
    # ----------------------------------------------------------
    def log_likelihood_obs(self, params, thresholds):
        '''
        Log-likelihood of OrderdModel for all observations.
        :return:
        '''
        wx = np.dot(self.exog, params)


        return
    # ----------------------------------------------------------

IndentationError: expected an indented block after function definition on line 41 (1870377951.py, line 44)